In [1]:
from tensorQ import *
from tqdm.notebook import tqdm

np.set_printoptions(precision=3, threshold=np.inf, suppress=True)

# Testing tn.py

## Testing site_svd

In [3]:
mps_site = np.random.rand(2,4,3)
l,r = site_svd(mps_site, which='left', max_dim=2)
l.shape, r.shape 

((2, 2, 3), (2, 4))

In [4]:
site = np.random.rand(2,4,3,3)

In [5]:
l, r = site_svd(site, which='left')
recon = ncon((l,r), ([-1,1,-3,-4],[1,-2]))
l.shape, r.shape, np.allclose(recon, site)

((2, 4, 3, 3), (4, 4), True)

In [6]:
l, r = site_svd(site, which='right')
recon = ncon((l,r), ([-1,1],[1,-2,-3,-4]))
l.shape, r.shape, np.allclose(recon, site)

((2, 2), (2, 4, 3, 3), True)

In [7]:
for msvr in [0.3,0.2,0.1]:
    l, r = site_svd(site, which='left', msvr=msvr)
    recon = ncon((l,r), ([-1,1,-3,-4],[1,-2]))
    print(l.shape, r.shape, np.linalg.norm(recon-site)) 

(2, 2, 3, 3) (2, 4) 1.5514305500228742
(2, 3, 3, 3) (3, 4) 0.9157992591600659
(2, 4, 3, 3) (4, 4) 2.759530691757413e-15


In [8]:
for max_dim in [1,2]:
    l, r = site_svd(site, which='right', max_dim=max_dim)
    recon = ncon((l,r), ([-1,1],[1,-2,-3,-4]))
    print(l.shape, r.shape, np.linalg.norm(recon-site)) 

(2, 1) (1, 4, 3, 3) 2.0121839057979645
(2, 2) (2, 4, 3, 3) 1.6016674704581467e-15


## Testing MPS overlap and compression 

In [9]:
mps = open_rmps(5,4)
mps.get_skeleton()

[(1, 2, 2), (2, 4, 2), (4, 4, 2), (4, 2, 2), (2, 1, 2)]

In [10]:
for max_dim in [4,3,2,1]:
    cmps = mps.compress(max_dim=max_dim)
    print(np.abs(mps.overlap(cmps))**2)

0.9999999999999978
0.963651488428629
0.8314668031589978
0.4035570620062541


In [11]:
for msvr in [0.5, 0.25, 0.1]:
    cmps = mps.compress(msvr=msvr)
    print(np.abs(mps.overlap(cmps))**2)

0.4035570620062541
0.9109904636136
0.9998133460319828


## Testing overlap, compose, add and compress for MPOs

In [2]:
mpo1 = rand_mpo(10,30)
mpo2 = rand_mpo(10,50)

In [3]:
(mpo1 @ mpo2).trace()

3.181542453404761e+25

In [4]:
mpo1.overlap(mpo2)

3.181542453404761e+25

In [5]:
# note that the two values are equal and that second takes significantly less time to execute

In [6]:
mpo1 = rand_mpo(4,3)
mpo2 = rand_mpo(4,5)
mpo3 = mpo1 + mpo2
np.allclose(mpo3.to_matrix(), (mpo1.to_matrix() + mpo2.to_matrix()))

True

In [7]:
# testing compression

mpo = rand_mpo(5,4)
mpo.get_skeleton()

[(1, 4, 2, 2), (4, 4, 2, 2), (4, 4, 2, 2), (4, 4, 2, 2), (4, 1, 2, 2)]

In [8]:
for max_dim in [4,3,2,1]:
    cmpo = mpo.compress(max_dim=max_dim)
    print(np.linalg.norm(mpo.to_matrix() - cmpo.to_matrix()))

7.54321440094149e-13
3.053508276970695
8.984096917471113
23.61717792536556


In [9]:
for msvr in [1e-1, 1e-2, 1e-3, 1e-4]:
    cmpo = mpo.compress(msvr=msvr)
    print(cmpo.get_max_dim(), np.linalg.norm(mpo.to_matrix() - cmpo.to_matrix()))

2 23.61717792536556
3 3.3016409047127557
4 7.54321440094149e-13
4 7.54321440094149e-13


## Testing MPDO methods 

In [6]:
# testing partial trace
circ = Circuit([H(0), CNOT([1,0], 2)])
mpdo = nt_sim(circ)
mpdo.to_matrix() 

array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]])

In [8]:
mpdo.partial_trace([0]).to_matrix()

array([[0.5, 0. ],
       [0. , 0.5]])

## Testing MPO initializations and decompositions

In [5]:
sites = [np.random.rand(1,4,2,2),np.random.rand(4,4,2,2),
         np.random.rand(4,1,2,2)]
mpo = MPO(sites)
mpo.to_matrix().shape

(8, 8)

In [6]:
mpo = all_zero_mpdo(2)
mpo.to_matrix()

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [7]:
data = np.random.rand(3,3,3,3,1,4)
a,b = decompose_site(data, min_sv_ratio=0.2)
a.shape, b.shape

((1, 4, 3, 3), (4, 4, 3, 3))

In [2]:
# testing decompose_three_site() 
contracted = np.random.rand(2,2,2,2,2,2,5,5)
top_site, middle_site, bottom_site = decompose_three_site(contracted) 
np.allclose(ncon((top_site, middle_site, bottom_site), ([-7,1,-1,-2],[1,2,-3,-4],[2,-8,-5,-6])), contracted) 

True

In [2]:
# testing decompose_four_site() 
contracted = np.random.rand(3,3,3,3,3,3,3,3,2,2)
first_site, second_site, third_site, fourth_site = decompose_four_site(contracted)
np.allclose(ncon((first_site, second_site, third_site, fourth_site), ([-9,1,-1,-2], [1,2,-3,-4], [2,3,-5,-6],
                                                          [3,-10,-7,-8])), contracted)

True

## Testing random MPS with various boundary conditions 

# Testing gates.py

In [2]:
raw_gate = np.array([[ 0.373-0.833j,  0.4  -0.08j ],
       [-0.256+0.318j,  0.911+0.054j]])

In [3]:
gate_b = Gate("Single qubit", 0, raw_gate.T)
gate_c = U(0, raw_gate.T)

In [4]:
gate_b.matrix / gate_c.matrix

array([[0.409-0.913j, 0.409-0.913j],
       [0.409-0.913j, 0.409-0.913j]])

In [5]:
nt_sim(Circuit([gate_b])).to_matrix()

array([[0.833+0.j   , 0.216-0.303j],
       [0.216+0.303j, 0.166+0.j   ]])

In [6]:
nt_sim(Circuit([gate_c])).to_matrix()

array([[0.833+0.j   , 0.216-0.304j],
       [0.216+0.304j, 0.167+0.j   ]])

In [7]:
gate_a = U01(0,0.5,0,1.5)
gate_b = CNOT3([0,1], 3)
gate_c = CNOT3([2,0], 3)
gate_d = CNOT3([0,2], 3)
gate_e = RZ01(0,0.5)

In [8]:
print(gate_a)
print(f"local dimension: {gate_a.get_local_dim()}")
print(f"superoperator shape: {gate_a.to_superop().shape}")
dec = gate_a.decompose()
print(f"ZSX decomposition: {[str(g) for g in dec]}")

print("")
print(f"matrix: {gate_a.matrix}")

U01 gate on site(s) [0] with angle(s) (0.5, 0, 1.5)
local dimension: 3
superoperator shape: (3, 3, 3, 3)
ZSX decomposition: ['RZ01 gate on site(s) [0] with angle(s) (1.5,)', 'SX01 gate on site(s) [0]', 'RZ01 gate on site(s) [0] with angle(s) (3.641592653589793,)', 'SX01 gate on site(s) [0]', 'RZ01 gate on site(s) [0] with angle(s) (3.141592653589793,)']

matrix: [[ 0.969+0.j    -0.018-0.247j  0.   +0.j   ]
 [ 0.247+0.j     0.069+0.966j  0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     1.   +0.j   ]]


In [9]:
print(gate_b)
print(gate_c)
print(gate_d)
print(gate_e)

print("")
print(f"matrix: {gate_e.matrix}")

top heavy CNOT3 gate on site(s) [0, 1]
top heavy CNOT3 gate on site(s) [2, 0]
bottom heavy CNOT3 gate on site(s) [2, 0]
RZ01 gate on site(s) [0] with angle(s) (0.5,)

matrix: [[1.   +0.j    0.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.878+0.479j 0.   +0.j   ]
 [0.   +0.j    0.   +0.j    1.   +0.j   ]]


In [15]:
# testing SWAP3 gate

gate_a = U01(0, 0.5,0,1.5)
gate_b = SX12(0)
gate_c = U01(1, 0.7,0.3,0.9)
gate_d = X12(1)
gate_e = SWAP3([0,1])

nt_sim(Circuit([gate_a, gate_b, gate_c, gate_d, gate_e]), debug=True)

1.0000 * (1.0000+0.0000j|00⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (1.0000+0.0000j|00⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (-0.7071+0.0000j|00⟩ + 0.0000+0.7071j|10⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.5000+0.0000j|00⟩ + 0.0000-0.5000j|01⟩ + 0.0000-0.5000j|10⟩ + -0.5000+0.0000j|11⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.5000+0.0000j|00⟩ + 0.0000-0.5000j|01⟩ + -0.2397+0.4388j|10⟩ + 0.4388+0.2397j|11⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.5000+0.0000j|00⟩ + -0.3221+0.3824j|01⟩ + -0.2397+0.4388j|10⟩ + -0.1812-0.4660j|11⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.6851+0.0000j|00⟩ + -0.4414+0.5240j|01⟩ + -0.1749-0.0000j|10⟩ + 0.1127-0.1338j|11⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.9102+0.0000j|00⟩ + -0.3322+0.0000j|01⟩ + -0.2324-0.0000j|10⟩ + 0.0848+0.0000j|11⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.9102+0.0000j|00⟩ + -0.3322-0.0000j|01⟩ + 0.2324-0.0000j|10⟩ + -0.0848+0.0000j|11⟩)
[(1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.9102+0.0000j|00⟩ + 0.3174+0.0982j|01⟩

## CUGate 

In [2]:
ccx = CUGate([1,2,0], CNOT([2,1],4), 4)
print(ccx)
ccx.matrix - (np.kron(np.array([[1,0],[0,0]]), np.eye(4)) + np.kron(np.array([[0,0],[0,1]]), CNOT([2,1], 4).matrix))

bottom heavy CU2 gate on site(s) [0 2 1]


array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [3]:
ccx = CUGate([0,1,2], CNOT([1,0],4), 4)
print(ccx)
ccx.matrix - (np.kron(np.eye(4), np.array([[1,0],[0,0]])) + np.kron(CNOT([2,1], 4).matrix, np.array([[0,0],[0,1]])))

top heavy CU2 gate on site(s) [0 1 2]


array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

# Testing noise.py

In [2]:
t1 = 100
t2 = 75
one_time = 40/1000 
two_time = 500/1000
coupling = 0.5
nm2 = QubitNoise(t1, t2, one_time, two_time, coupling)
nm3 = QutritNoise(t1, t2, one_time, two_time, coupling)

In [3]:
k_list = nm2.get_kraus(one_time)
sum(k.conj().T@k for k in k_list)

array([[1., 0.],
       [0., 1.]])

In [4]:
k_list = nm3.get_kraus(one_time)
sum(k.conj().T@k for k in k_list)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

# Testing circuits.py

## Qubits 

In [2]:
t1 = 100
t2 = 75
one_time = 40/1000 
two_time = 500/1000
coupling = 0.5
nm = QubitNoise(t1, t2, one_time, two_time, coupling)

In [3]:
gate_list = [U(0,1.34,-np.pi,0), CNOT([1,0], 2), U(1, np.pi/2,0,np.pi), U(2,1.89, np.pi, 0), SX(2), 
             CNOT([2,1], 2), U(1,2.82,0,np.pi), U(2,1.89, np.pi, 0), CNOT([2,1], 2)]
circ = Circuit(gate_list)
print(circ)

U gate on site(s) [0] with angle(s) (1.34, -3.141592653589793, 0) 
bottom heavy CNOT2 gate on site(s) [0, 1] 
U gate on site(s) [1] with angle(s) (1.5707963267948966, 0, 3.141592653589793) 
U gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 
SX gate on site(s) [2] 
bottom heavy CNOT2 gate on site(s) [1, 2] 
U gate on site(s) [1] with angle(s) (2.82, 0, 3.141592653589793) 
U gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 
bottom heavy CNOT2 gate on site(s) [1, 2]


In [4]:
circ.local_dim

2

In [5]:
# construct layers without decomposing U gates
circ.construct_layers()
print(circ)

U gate on site(s) [0] with angle(s) (1.34, -3.141592653589793, 0) 
U gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 

bottom heavy CNOT2 gate on site(s) [0, 1] 
SX gate on site(s) [2] 

U gate on site(s) [1] with angle(s) (1.5707963267948966, 0, 3.141592653589793) 

bottom heavy CNOT2 gate on site(s) [1, 2] 

U gate on site(s) [1] with angle(s) (2.82, 0, 3.141592653589793) 
U gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 

bottom heavy CNOT2 gate on site(s) [1, 2]


In [6]:
print(circ.build(nm))

RZ gate on site(s) [0] with angle(s) (0,) 
RZ gate on site(s) [2] with angle(s) (0,) 
Idling on site(s) [0] for time 0.0 
Idling on site(s) [1] for time 0.0 
Idling on site(s) [2] for time 0.0 

SX gate on site(s) [0] 
SX gate on site(s) [2] 
Idling on site(s) [0] for time 0.04 
Idling on site(s) [1] for time 0.02 
Idling on site(s) [2] for time 0.04 

RZ gate on site(s) [0] with angle(s) (4.481592653589793,) 
RZ gate on site(s) [2] with angle(s) (5.031592653589793,) 
Idling on site(s) [0] for time 0.0 
Idling on site(s) [1] for time 0.0 
Idling on site(s) [2] for time 0.0 

SX gate on site(s) [0] 
SX gate on site(s) [2] 
Idling on site(s) [0] for time 0.04 
Idling on site(s) [1] for time 0.02 
Idling on site(s) [2] for time 0.04 

RZ gate on site(s) [0] with angle(s) (0.0,) 
RZ gate on site(s) [2] with angle(s) (6.283185307179586,) 
Idling on site(s) [0] for time 0.0 
Idling on site(s) [1] for time 0.0 
Idling on site(s) [2] for time 0.0 

bottom heavy CNOT2 gate on site(s) [0, 1] 
SX

## Qutrits 

In [5]:
t1 = 100
t2 = 75
one_time = 40/1000 
two_time = 500/1000
coupling = 0.5
nm = QutritNoise(t1, t2, one_time, two_time, coupling)

In [7]:
gate_list = [U01(0,1.34,-np.pi,0), CNOT3([1,0], 2), U01(1, np.pi/2,0,np.pi), U01(2,1.89, np.pi, 0), SX01(2), 
             CNOT3([2,1], 2), U01(1,2.82,0,np.pi), U01(2,1.89, np.pi, 0), CNOT3([2,1], 2)]
circ = Circuit(gate_list)
print(circ)

U01 gate on site(s) [0] with angle(s) (1.34, -3.141592653589793, 0) 
bottom heavy CNOT3 gate on site(s) [0, 1] 
U01 gate on site(s) [1] with angle(s) (1.5707963267948966, 0, 3.141592653589793) 
U01 gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 
SX01 gate on site(s) [2] 
bottom heavy CNOT3 gate on site(s) [1, 2] 
U01 gate on site(s) [1] with angle(s) (2.82, 0, 3.141592653589793) 
U01 gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 
bottom heavy CNOT3 gate on site(s) [1, 2]


In [8]:
circ.local_dim

3

In [9]:
# construct layers without decomposing U gates
circ.construct_layers()
print(circ)

U01 gate on site(s) [0] with angle(s) (1.34, -3.141592653589793, 0) 
U01 gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 

bottom heavy CNOT3 gate on site(s) [0, 1] 
SX01 gate on site(s) [2] 

U01 gate on site(s) [1] with angle(s) (1.5707963267948966, 0, 3.141592653589793) 

bottom heavy CNOT3 gate on site(s) [1, 2] 

U01 gate on site(s) [1] with angle(s) (2.82, 0, 3.141592653589793) 
U01 gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 

bottom heavy CNOT3 gate on site(s) [1, 2]


In [10]:
# circuit after decomposing U and adding noise
print(circ.build(nm))

U01 gate on site(s) [0] with angle(s) (1.34, -3.141592653589793, 0) 
U01 gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 

bottom heavy CNOT3 gate on site(s) [0, 1] 
SX01 gate on site(s) [2] 

U01 gate on site(s) [1] with angle(s) (1.5707963267948966, 0, 3.141592653589793) 

bottom heavy CNOT3 gate on site(s) [1, 2] 

U01 gate on site(s) [1] with angle(s) (2.82, 0, 3.141592653589793) 
U01 gate on site(s) [2] with angle(s) (1.89, 3.141592653589793, 0) 

bottom heavy CNOT3 gate on site(s) [1, 2]


# Testing nt_sim.py 

In [5]:
gate_a = U(0,0.5,0,1.5)
gate_b = CNOT([2,0], 3)
circ = Circuit([gate_a, gate_b])

mpdo = nt_sim(circ, dm=True, debug=True)

  0%|          | 0/6 [00:00<?, ?it/s]

1.0000 * (1.0000+0.0000j|000⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (-0.7071+0.0000j|000⟩ + 0.0000+0.7071j|100⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (0.7071+0.0000j|000⟩ + -0.3390+0.6205j|100⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (0.9689+0.0000j|000⟩ + -0.2474-0.0000j|100⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (0.9689+0.0000j|000⟩ + -0.0175-0.2468j|100⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (-0.9689+0.0000j|000⟩ + 0.0175+0.2468j|101⟩)
[(4, 1, 2, 2), (1, 1, 2, 2), (1, 4, 2, 2)]



In [4]:
gate_a = U01(0,0.5,0,1.5)
gate_b = CNOT3([2,0], 3)
circ = Circuit([gate_a, gate_b])

mpdo = nt_sim(circ, dm=True, debug=True)

  0%|          | 0/6 [00:00<?, ?it/s]

1.0000 * (1.0000+0.0000j|000⟩)
[(1, 1, 3, 3), (1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (-0.7071+0.0000j|000⟩ + 0.0000+0.7071j|100⟩)
[(1, 1, 3, 3), (1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.7071+0.0000j|000⟩ + -0.3390+0.6205j|100⟩)
[(1, 1, 3, 3), (1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.9689+0.0000j|000⟩ + -0.2474-0.0000j|100⟩)
[(1, 1, 3, 3), (1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (0.9689+0.0000j|000⟩ + -0.0175-0.2468j|100⟩)
[(1, 1, 3, 3), (1, 1, 3, 3), (1, 1, 3, 3)]

1.0000 * (-0.9689+0.0000j|000⟩ + 0.0175+0.2468j|101⟩)
[(9, 1, 3, 3), (1, 1, 3, 3), (1, 9, 3, 3)]



In [3]:
# Test the action of a CCX gate 
ccx = CUGate([1,2,0], CNOT([2,1],4), 4)
print(ccx)

gate_a = X(0)
gate_b = X(1)
gate_c = ccx
circ = Circuit([gate_a, gate_b, ccx])

mpdo = nt_sim(circ, dm=True, debug=True)

bottom heavy CU2 gate on site(s) [0 1 2]
1.0000 * (1.0000|100⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (1.0000|110⟩)
[(1, 1, 2, 2), (1, 1, 2, 2), (1, 1, 2, 2)]

1.0000 * (1.0000|111⟩)
[(1, 4, 2, 2), (4, 4, 2, 2), (4, 1, 2, 2)]



In [6]:
ccx.matrix.shape 

(8, 8)

In [3]:
# test four_local()

circ = Circuit([H01(0), CNOT3([1,0], 4), CNOT3([2,1], 4), CNOT3([3,2], 4)], 4)
pretty_print_sv(circ.to_matrix() @ string_to_sv('0000', 3), 3) 

g = Gate("four-local", [0,1,2,3], circ.to_matrix())
new_circ = Circuit([g], 4)
pretty_print_dm(nt_sim(new_circ, dm=True), 3) 

0.7071+0.0000j|0000⟩ + 0.7071+0.0000j|1111⟩
1.0000 * (0.7071+0.0000j|0000⟩ + 0.7071+0.0000j|1111⟩)


# Testing utils.py

In [19]:
rho1 = rand_dm(4)
rho2 = rand_dm(4)

In [20]:
1-np.trace(rho1@rho1)

(0.3635584285840525+0j)

In [21]:
fidelity(rho1, rho2)

0.8622027287979752

In [22]:
two_norm_fidelity(rho1, rho2)

0.5842163416931787